### Script to plot mesh spacing for variable-resolution global MPAS mesh ###

---

Manda Chasteen
<br>
April 2024

In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mp
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# My colormaps file:
import custom_colormaps as cc

np.set_printoptions(suppress=True)

import warnings
warnings.simplefilter("ignore") 

In [ ]:
grid_dir = ''                                           # Add directory here
grid_file = 'x5.6488066.static.nc'

f = xr.open_dataset(grid_dir+grid_file).squeeze()

In [ ]:
latCell = f.latCell * 180.0 / np.pi
lonCell = f.lonCell * 180.0 / np.pi

meshDensity = f.meshDensity
dc = 3.0 * np.power(1.0 / meshDensity, 0.25)

In [ ]:
### Plot mesh:

plt.rcParams['savefig.dpi'] = 250
proj = ccrs.NearsidePerspective(central_longitude=-105.0, central_latitude=40.0)
fig, ax = plt.subplots(1,1,figsize=(10,10),subplot_kw={'projection': proj}) 
fig.patch.set_facecolor('white')
fig.patch.set_alpha(1)
ax.set_global()

#######
gl_color = 'xkcd:gunmetal'
cl_color = 'xkcd:darkblue' 
cl_lwidth = 1.0
kw_clabels = {'fontsize': 8.5, 'inline': True, 'inline_spacing': 3, 'fmt': '%i',
              'rightside_up': True, 'use_clabeltext': True, 'zorder' : 3.5}

ax.coastlines(resolution='50m', color=cl_color, linewidth=cl_lwidth, zorder=1, alpha=0.8)
gl = ax.gridlines(xlocs=np.arange(-180,180,30), ylocs=[0,30,60,90], linewidths=0.1, linestyles=[(0, (2.0, 2.0))], color=gl_color, zorder=2, alpha=0.3)
# gl.xlabels_bottom = True
# gl.xlabel_style = {'size': 8.5}

# Set colormap and norm:
cmap = cc.deep_sea().reversed()
vmin = 3
vmax = 15
norm = mp.colors.Normalize(vmin=vmin, vmax=vmax)

cn = ax.tricontourf(lonCell, latCell, dc, vmin=vmin, vmax=vmax, levels=np.linspace(3,15,100), extend='max', cmap=cmap, norm=norm, transform=ccrs.PlateCarree(), zorder=0)
cbar = fig.colorbar(cn, ax=ax, orientation="horizontal",extend='max',shrink=0.7,pad=0.015,fraction=0.035,aspect=35,ticks = np.arange(3,16,1))

c0 = ax.tricontour(lonCell, latCell, dc, [3.01, 4.0, 6.0, 9.0, 12.0], linewidths=1.5, colors='black', transform=ccrs.PlateCarree(), zorder=3)
lab0 = ax.clabel(c0, c0.levels, **kw_clabels)
plt.setp(lab0, path_effects=[pe.withStroke(linewidth=1.5, foreground="w")])

pngname = 'mpas_15to3km_mesh.png'
fig.savefig(pngname,bbox_inches="tight")